In [1]:
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
from pyspark.sql.functions import lit

from datetime import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA

# import plotly.plotly as py
# import plotly.graph_objs as go
import pandas as pd
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [2]:
crimes = spark.sql("select * from crimes___2001_to_present_9be1b_csv")
display(crimes.select("*"))

ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
10000092,HY189866,03/18/2015 07:44:00 PM,047XX W OHIO ST,041A,BATTERY,AGGRAVATED: HANDGUN,STREET,false,false,1111,011,28,25,04B,1144606,1903566,2015,02/10/2018 03:50:01 PM,41.891398861,-87.744384567,"(41.891398861, -87.744384567)"
10000094,HY190059,03/18/2015 11:00:00 PM,066XX S MARSHFIELD AVE,4625,OTHER OFFENSE,PAROLE VIOLATION,STREET,true,false,0725,007,15,67,26,1166468,1860715,2015,02/10/2018 03:50:01 PM,41.773371528,-87.665319468,"(41.773371528, -87.665319468)"
10000095,HY190052,03/18/2015 10:45:00 PM,044XX S LAKE PARK AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,false,true,0222,002,4,39,08B,1185075,1875622,2015,02/10/2018 03:50:01 PM,41.81386068,-87.596642837,"(41.81386068, -87.596642837)"
10000096,HY190054,03/18/2015 10:30:00 PM,051XX S MICHIGAN AVE,0460,BATTERY,SIMPLE,APARTMENT,false,false,0225,002,3,40,08B,1178033,1870804,2015,02/10/2018 03:50:01 PM,41.800802415,-87.622619343,"(41.800802415, -87.622619343)"
10000097,HY189976,03/18/2015 09:00:00 PM,047XX W ADAMS ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,false,false,1113,011,28,25,03,1144920,1898709,2015,02/10/2018 03:50:01 PM,41.878064761,-87.743354013,"(41.878064761, -87.743354013)"
10000098,HY190032,03/18/2015 10:00:00 PM,049XX S DREXEL BLVD,0460,BATTERY,SIMPLE,APARTMENT,false,false,0223,002,4,39,08B,1183018,1872537,2015,02/10/2018 03:50:01 PM,41.805443345,-87.604283976,"(41.805443345, -87.604283976)"
10000099,HY190047,03/18/2015 11:00:00 PM,070XX S MORGAN ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,false,true,0733,007,17,68,08B,1170859,1858210,2015,02/10/2018 03:50:01 PM,41.766402779,-87.649296123,"(41.766402779, -87.649296123)"
10000100,HY189988,03/18/2015 09:35:00 PM,042XX S PRAIRIE AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,false,true,0213,002,3,38,08B,1178746,1876914,2015,02/10/2018 03:50:01 PM,41.817552577,-87.619818523,"(41.817552577, -87.619818523)"
10000101,HY190020,03/18/2015 10:09:00 PM,036XX S WOLCOTT AVE,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,STREET,true,false,0912,009,11,59,18,1164279,1880656,2015,02/10/2018 03:50:01 PM,41.828138428,-87.672782106,"(41.828138428, -87.672782106)"
10000104,HY189964,03/18/2015 09:25:00 PM,097XX S PRAIRIE AVE,0460,BATTERY,SIMPLE,RESIDENCE PORCH/HALLWAY,false,false,0511,005,6,49,08B,1179637,1840444,2015,02/10/2018 03:50:01 PM,41.71745472,-87.617663257,"(41.71745472, -87.617663257)"


In [3]:
from datetime import datetime
from pyspark.sql.functions import to_timestamp,to_date
crimes= crimes.withColumn("new_date",to_date(crimes.Date, "mm/dd/yyyy HH:mm:ss"))

display(crimes.select('*'))

ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Crime_Count,new_date
10000092,HY189866,03/18/2015 07:44:00 PM,047XX W OHIO ST,041A,BATTERY,AGGRAVATED: HANDGUN,STREET,false,false,1111,011,28,25,04B,1144606,1903566,2015,02/10/2018 03:50:01 PM,41.891398861,-87.744384567,"(41.891398861, -87.744384567)",1,2015-01-18
10000094,HY190059,03/18/2015 11:00:00 PM,066XX S MARSHFIELD AVE,4625,OTHER OFFENSE,PAROLE VIOLATION,STREET,true,false,0725,007,15,67,26,1166468,1860715,2015,02/10/2018 03:50:01 PM,41.773371528,-87.665319468,"(41.773371528, -87.665319468)",1,2015-01-18
10000095,HY190052,03/18/2015 10:45:00 PM,044XX S LAKE PARK AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,false,true,0222,002,4,39,08B,1185075,1875622,2015,02/10/2018 03:50:01 PM,41.81386068,-87.596642837,"(41.81386068, -87.596642837)",1,2015-01-18
10000096,HY190054,03/18/2015 10:30:00 PM,051XX S MICHIGAN AVE,0460,BATTERY,SIMPLE,APARTMENT,false,false,0225,002,3,40,08B,1178033,1870804,2015,02/10/2018 03:50:01 PM,41.800802415,-87.622619343,"(41.800802415, -87.622619343)",1,2015-01-18
10000097,HY189976,03/18/2015 09:00:00 PM,047XX W ADAMS ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,false,false,1113,011,28,25,03,1144920,1898709,2015,02/10/2018 03:50:01 PM,41.878064761,-87.743354013,"(41.878064761, -87.743354013)",1,2015-01-18
10000098,HY190032,03/18/2015 10:00:00 PM,049XX S DREXEL BLVD,0460,BATTERY,SIMPLE,APARTMENT,false,false,0223,002,4,39,08B,1183018,1872537,2015,02/10/2018 03:50:01 PM,41.805443345,-87.604283976,"(41.805443345, -87.604283976)",1,2015-01-18
10000099,HY190047,03/18/2015 11:00:00 PM,070XX S MORGAN ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,false,true,0733,007,17,68,08B,1170859,1858210,2015,02/10/2018 03:50:01 PM,41.766402779,-87.649296123,"(41.766402779, -87.649296123)",1,2015-01-18
10000100,HY189988,03/18/2015 09:35:00 PM,042XX S PRAIRIE AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,false,true,0213,002,3,38,08B,1178746,1876914,2015,02/10/2018 03:50:01 PM,41.817552577,-87.619818523,"(41.817552577, -87.619818523)",1,2015-01-18
10000101,HY190020,03/18/2015 10:09:00 PM,036XX S WOLCOTT AVE,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,STREET,true,false,0912,009,11,59,18,1164279,1880656,2015,02/10/2018 03:50:01 PM,41.828138428,-87.672782106,"(41.828138428, -87.672782106)",1,2015-01-18
10000104,HY189964,03/18/2015 09:25:00 PM,097XX S PRAIRIE AVE,0460,BATTERY,SIMPLE,RESIDENCE PORCH/HALLWAY,false,false,0511,005,6,49,08B,1179637,1840444,2015,02/10/2018 03:50:01 PM,41.71745472,-87.617663257,"(41.71745472, -87.617663257)",1,2015-01-18


In [4]:
from pyspark.sql.functions import year, month, dayofmonth, hour, dayofweek
crimes= crimes.withColumn('hour', hour(crimes['new_date']))
crimes=crimes.withColumn("Month",month(crimes["new_date"]))
crimes=crimes.withColumn("New_Year",year(crimes["new_date"]))
crimes=crimes.withColumn('week_day', dayofweek(crimes['new_date']))
display(crimes.select('*'))

ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Crime_Count,new_date,hour,Month,New_Year,week_day
10000092,HY189866,03/18/2015 07:44:00 PM,047XX W OHIO ST,041A,BATTERY,AGGRAVATED: HANDGUN,STREET,false,false,1111,011,28,25,04B,1144606,1903566,2015,02/10/2018 03:50:01 PM,41.891398861,-87.744384567,"(41.891398861, -87.744384567)",1,2015-01-18,0,1,2015,1
10000094,HY190059,03/18/2015 11:00:00 PM,066XX S MARSHFIELD AVE,4625,OTHER OFFENSE,PAROLE VIOLATION,STREET,true,false,0725,007,15,67,26,1166468,1860715,2015,02/10/2018 03:50:01 PM,41.773371528,-87.665319468,"(41.773371528, -87.665319468)",1,2015-01-18,0,1,2015,1
10000095,HY190052,03/18/2015 10:45:00 PM,044XX S LAKE PARK AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,false,true,0222,002,4,39,08B,1185075,1875622,2015,02/10/2018 03:50:01 PM,41.81386068,-87.596642837,"(41.81386068, -87.596642837)",1,2015-01-18,0,1,2015,1
10000096,HY190054,03/18/2015 10:30:00 PM,051XX S MICHIGAN AVE,0460,BATTERY,SIMPLE,APARTMENT,false,false,0225,002,3,40,08B,1178033,1870804,2015,02/10/2018 03:50:01 PM,41.800802415,-87.622619343,"(41.800802415, -87.622619343)",1,2015-01-18,0,1,2015,1
10000097,HY189976,03/18/2015 09:00:00 PM,047XX W ADAMS ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,false,false,1113,011,28,25,03,1144920,1898709,2015,02/10/2018 03:50:01 PM,41.878064761,-87.743354013,"(41.878064761, -87.743354013)",1,2015-01-18,0,1,2015,1
10000098,HY190032,03/18/2015 10:00:00 PM,049XX S DREXEL BLVD,0460,BATTERY,SIMPLE,APARTMENT,false,false,0223,002,4,39,08B,1183018,1872537,2015,02/10/2018 03:50:01 PM,41.805443345,-87.604283976,"(41.805443345, -87.604283976)",1,2015-01-18,0,1,2015,1
10000099,HY190047,03/18/2015 11:00:00 PM,070XX S MORGAN ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,false,true,0733,007,17,68,08B,1170859,1858210,2015,02/10/2018 03:50:01 PM,41.766402779,-87.649296123,"(41.766402779, -87.649296123)",1,2015-01-18,0,1,2015,1
10000100,HY189988,03/18/2015 09:35:00 PM,042XX S PRAIRIE AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,false,true,0213,002,3,38,08B,1178746,1876914,2015,02/10/2018 03:50:01 PM,41.817552577,-87.619818523,"(41.817552577, -87.619818523)",1,2015-01-18,0,1,2015,1
10000101,HY190020,03/18/2015 10:09:00 PM,036XX S WOLCOTT AVE,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,STREET,true,false,0912,009,11,59,18,1164279,1880656,2015,02/10/2018 03:50:01 PM,41.828138428,-87.672782106,"(41.828138428, -87.672782106)",1,2015-01-18,0,1,2015,1
10000104,HY189964,03/18/2015 09:25:00 PM,097XX S PRAIRIE AVE,0460,BATTERY,SIMPLE,RESIDENCE PORCH/HALLWAY,false,false,0511,005,6,49,08B,1179637,1840444,2015,02/10/2018 03:50:01 PM,41.71745472,-87.617663257,"(41.71745472, -87.617663257)",1,2015-01-18,0,1,2015,1


In [5]:
crimes = crimes.withColumn("Crime_Count", lit(1))
crimes.show()


+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-----------+
 ID|Case Number| Date| Block|IUCR| Primary Type| Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year| Updated On| Latitude| Longitude| Location|Crime_Count|
+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-----------+
10000092| HY189866|03/18/2015 07:44:...| 047XX W OHIO ST|041A| BATTERY| AGGRAVATED: HANDGUN| STREET| false| false|1111| 011| 28| 25| 04B| 1144606| 1903566|2015|02/10/2018 03:50:...|41.891398861|-87.744384567|(41.891398861, -8...| 1|
10000094| HY190059|03/18/2015 11:00:...|066XX S MARSHFIEL...|4625| OTHER OFFENSE| PAROLE VIOLATION| STREET| true| false|0725| 007| 15| 67| 26| 1166468| 1860715|2015|02/10/2018 03:50:...|41.773371528|-87.665319468|(41.773371528, -8...| 1|
10000095| HY190052|03/18/2015 10:45:...|044XX S LAKE PARK...|0486| BATTERY|DOMESTIC BATTERY ...| APARTMENT| false| true|0222| 002| 4| 39| 08B| 1185075| 1875622|2015|02/10/2018 03:50:...| 41.81386068|-87.596642837|(41.81386068, -87...| 1|
10000096| HY190054|03/18/2015 10:30:...|051XX S MICHIGAN AVE|0460| BATTERY| SIMPLE| APARTMENT| false| false|0225| 002| 3| 40| 08B| 1178033| 1870804|2015|02/10/2018 03:50:...|41.800802415|-87.622619343|(41.800802415, -8...| 1|
10000097| HY189976|03/18/2015 09:00:...| 047XX W ADAMS ST|031A| ROBBERY| ARMED: HANDGUN| SIDEWALK| false| false|1113| 011| 28| 25| 03| 1144920| 1898709|2015|02/10/2018 03:50:...|41.878064761|-87.743354013|(41.878064761, -8...| 1|
10000098| HY190032|03/18/2015 10:00:...| 049XX S DREXEL BLVD|0460| BATTERY| SIMPLE| APARTMENT| false| false|0223| 002| 4| 39| 08B| 1183018| 1872537|2015|02/10/2018 03:50:...|41.805443345|-87.604283976|(41.805443345, -8...| 1|
10000099| HY190047|03/18/2015 11:00:...| 070XX S MORGAN ST|0486| BATTERY|DOMESTIC BATTERY ...| APARTMENT| false| true|0733| 007| 17| 68| 08B| 1170859| 1858210|2015|02/10/2018 03:50:...|41.766402779|-87.649296123|(41.766402779, -8...| 1|
10000100| HY189988|03/18/2015 09:35:...| 042XX S PRAIRIE AVE|0486| BATTERY|DOMESTIC BATTERY ...| APARTMENT| false| true|0213| 002| 3| 38| 08B| 1178746| 1876914|2015|02/10/2018 03:50:...|41.817552577|-87.619818523|(41.817552577, -8...| 1|
10000101| HY190020|03/18/2015 10:09:...| 036XX S WOLCOTT AVE|1811| NARCOTICS|POSS: CANNABIS 30...| STREET| true| false|0912| 009| 11| 59| 18| 1164279| 1880656|2015|02/10/2018 03:50:...|41.828138428|-87.672782106|(41.828138428, -8...| 1|
10000104| HY189964|03/18/2015 09:25:...| 097XX S PRAIRIE AVE|0460| BATTERY| SIMPLE|RESIDENCE PORCH/H...| false| false|0511| 005| 6| 49| 08B| 1179637| 1840444|2015|02/10/2018 03:50:...| 41.71745472|-87.617663257|(41.71745472, -87...| 1|
10000105| HY189984|03/18/2015 09:30:...|130XX S DR MARTIN...|1320| CRIMINAL DAMAGE| TO VEHICLE|PARKING LOT/GARAG...| false| false|0533| 005| 9| 54| 14| 1180907| 1818839|2015|02/10/2018 03:50:...|41.658138493|-87.613672862|(41.658138493, -8...| 1|
10000108| HY189719|03/15/2015 04:10:...|078XX S VINCENNES...|2825| OTHER OFFENSE|HARASSMENT BY TEL...|CTA GARAGE / OTHE...| false| true|0623| 006| 17| 69| 26| 1175130| 1853144|2015|02/10/2018 03:50:...|41.752406801|-87.633792381|(41.752406801, -8...| 1|
10000109| HY189966|03/18/2015 09:14:...|086XX S EXCHANGE AVE|143A| WEAPONS VIOLATION|UNLAWFUL POSS OF ...|DRIVEWAY - RESIDE...| true| false|0423| 004| 10| 46| 15| 1197309| 1848290|2015|02/10/2018 03:50:...|41.738563465|-87.552678593|(41.738563465, -8...| 1|
10000110| HY190056|03/18/2015 10:50:...| 014XX S ASHLAND AVE|0460|

In [6]:
crimes.count()

Out[ 11 ]: 6657414

In [7]:
drop_na=crimes.na.drop()

In [8]:
drop_na.describe(['Crime_Count']).show()

+-------+-----------+
summary|Crime_Count|
+-------+-----------+
 count| 5988839|
 mean| 1.0|
 stddev| 0.0|
 min| 1|
 max| 1|
+-------+-----------+

In [9]:
drop_na.count()

Out[ 16 ]: 5988839

In [10]:
crime_pivot=crimes.groupBy("Year").pivot("Primary Type").sum("Crime_Count").orderBy("Year", ascending=True)
crime_pivot.show()

+----+-----+-------+-------+--------+---------------------------------+-------------------+---------------+-----------------+------------------+-----------------+--------+--------+-----------------+--------------------------------+------------+----------+--------------------+-------------------+---------+--------------+------------+--------------------------------+---------+--------------------------+------------------------+-------------+------------+----------------+----------------------+---------+-------+-----------+--------+-----+-----------------+
Year|ARSON|ASSAULT|BATTERY|BURGLARY|CONCEALED CARRY LICENSE VIOLATION|CRIM SEXUAL ASSAULT|CRIMINAL DAMAGE|CRIMINAL TRESPASS|DECEPTIVE PRACTICE|DOMESTIC VIOLENCE|GAMBLING|HOMICIDE|HUMAN TRAFFICKING|INTERFERENCE WITH PUBLIC OFFICER|INTIMIDATION|KIDNAPPING|LIQUOR LAW VIOLATION|MOTOR VEHICLE THEFT|NARCOTICS|NON - CRIMINAL|NON-CRIMINAL|NON-CRIMINAL (SUBJECT SPECIFIED)|OBSCENITY|OFFENSE INVOLVING CHILDREN|OTHER NARCOTIC VIOLATION|OTHER OFFENSE|PROSTITUTION|PUBLIC INDECENCY|PUBLIC PEACE VIOLATION|RITUALISM|ROBBERY|SEX OFFENSE|STALKING|THEFT|WEAPONS VIOLATION|
+----+-----+-------+-------+--------+---------------------------------+-------------------+---------------+-----------------+------------------+-----------------+--------+--------+-----------------+--------------------------------+------------+----------+--------------------+-------------------+---------+--------------+------------+--------------------------------+---------+--------------------------+------------------------+-------------+------------+----------------+----------------------+---------+-------+-----------+--------+-----+-----------------+
2001| 1010| 31384| 93447| 26011| null| 1796| 55851| 13240| 14904| 1| 934| 667| null| 406| 279| 933| 1637| 27549| 50567| null| null| null| 19| 2240| 6| 29657| 6026| 9| 2750| 8| 18441| 2231| 203|99264| 4274|
2002| 1032| 31521| 94152| 25623| null| 1830| 55940| 13880| 13715| null| 971| 657| null| 361| 337| 829| 1414| 25121| 51789| null| null| null| 26| 2576| 5| 32599| 6408| 8| 2457| 2| 18522| 2167| 200|98327| 4281|
2003| 955| 29477| 88377| 25156| null| 1595| 55011| 14807| 13451| null| 1088| 604| null| 408| 364| 705| 1311| 22748| 54288| null| 1| null| 16| 3033| 2| 31147| 6214| 6| 2430| 2| 17332| 2071| 247|98875| 4211|
2004| 778| 28850| 87135| 24564| null| 1570| 53164| 15913| 13243| null| 1122| 454| null| 531| 349| 481| 985| 22805| 57060| null| null| null| 13| 3078| 9| 29532| 7476| 9| 2495| 1| 15978| 1804| 215|95463| 4297|
2005| 691| 27066| 83965| 25504| null| 1546| 54548| 16655| 13562| null| 1078| 453| null| 615| 258| 389| 1005| 22497| 56234| null| null| null| 19| 2881| 9| 28028| 6124| 4| 2730| 2| 16047| 1805| 192|85685| 4106|
2006| 726| 25945| 80666| 24324| null| 1466| 57124| 14505| 13577| null| 1368| 476| null| 758| 276| 339| 1135| 21818| 55813| null| null| null| 17| 2753| 9| 27100| 7034| 4| 3068| 7| 15968| 1564| 186|86240| 3821|
2007| 712| 26314| 79591| 24858| null| 1538| 53749| 13699| 14137| null| 1409| 447| null| 677| 255| 330| 1170| 18573| 54454| null| null| null| 11| 2880| 12| 26863| 6087| 5| 3315| 1| 15450| 1526| 213|85156| 3554|
2008| 644| 25447| 75922| 26218| null| 1525| 52841| 12310| 14879| null| 1199| 513| null| 580| 261| 359| 912| 18881| 46507| null| null| null| 13| 2618| 12| 26533| 5141| 4| 3013| null| 16703| 1490| 190|88432| 3877|
2009| 616| 22861| 68462| 26766| null| 1414| 47724| 10851| 13805| null| 991| 460| null| 573| 231| 293| 746| 15482| 43543| null| 1| null| 21| 2574| 7| 25601| 3940| 10| 3147| null| 15980| 1251| 167|80972| 4158|
2010| 522| 21536| 65404| 26422| null| 1351| 40653| 9401| 12488| null| 927| 438| null| 797| 197| 313| 736| 19029| 43393| null| 1| null| 34| 2545| 6| 22013| 2485| 7| 3538| null| 14274| 1115| 189|76754| 3704|
2011| 504| 20411| 60458| 26619| null| 1477| 37332| 8659| 12616| null| 736| 437| null| 1048| 171| 266| 619| 19388| 38605| null| null| null| 40| 2343| 5| 20191| 2424| 13| 3095| null| 13983| 1073| 181|75148| 3880|
2012| 469| 19898| 59135| 22

In [11]:
display(crime_pivot.select("*"))

Year,ARSON,ASSAULT,BATTERY,BURGLARY,CONCEALED CARRY LICENSE VIOLATION,CRIM SEXUAL ASSAULT,CRIMINAL DAMAGE,CRIMINAL TRESPASS,DECEPTIVE PRACTICE,DOMESTIC VIOLENCE,GAMBLING,HOMICIDE,HUMAN TRAFFICKING,INTERFERENCE WITH PUBLIC OFFICER,INTIMIDATION,KIDNAPPING,LIQUOR LAW VIOLATION,MOTOR VEHICLE THEFT,NARCOTICS,NON - CRIMINAL,NON-CRIMINAL,NON-CRIMINAL (SUBJECT SPECIFIED),OBSCENITY,OFFENSE INVOLVING CHILDREN,OTHER NARCOTIC VIOLATION,OTHER OFFENSE,PROSTITUTION,PUBLIC INDECENCY,PUBLIC PEACE VIOLATION,RITUALISM,ROBBERY,SEX OFFENSE,STALKING,THEFT,WEAPONS VIOLATION
2001,1010,31384,93447,26011,null,1796,55851,13240,14904,1,934,667,null,406,279,933,1637,27549,50567,null,null,null,19,2240,6,29657,6026,9,2750,8,18441,2231,203,99264,4274
2002,1032,31521,94152,25623,null,1830,55940,13880,13715,null,971,657,null,361,337,829,1414,25121,51789,null,null,null,26,2576,5,32599,6408,8,2457,2,18522,2167,200,98327,4281
2003,955,29477,88377,25156,null,1595,55011,14807,13451,null,1088,604,null,408,364,705,1311,22748,54288,null,1,null,16,3033,2,31147,6214,6,2430,2,17332,2071,247,98875,4211
2004,778,28850,87135,24564,null,1570,53164,15913,13243,null,1122,454,null,531,349,481,985,22805,57060,null,null,null,13,3078,9,29532,7476,9,2495,1,15978,1804,215,95463,4297
2005,691,27066,83965,25504,null,1546,54548,16655,13562,null,1078,453,null,615,258,389,1005,22497,56234,null,null,null,19,2881,9,28028,6124,4,2730,2,16047,1805,192,85685,4106
2006,726,25945,80666,24324,null,1466,57124,14505,13577,null,1368,476,null,758,276,339,1135,21818,55813,null,null,null,17,2753,9,27100,7034,4,3068,7,15968,1564,186,86240,3821
2007,712,26314,79591,24858,null,1538,53749,13699,14137,null,1409,447,null,677,255,330,1170,18573,54454,null,null,null,11,2880,12,26863,6087,5,3315,1,15450,1526,213,85156,3554
2008,644,25447,75922,26218,null,1525,52841,12310,14879,null,1199,513,null,580,261,359,912,18881,46507,null,null,null,13,2618,12,26533,5141,4,3013,null,16703,1490,190,88432,3877
2009,616,22861,68462,26766,null,1414,47724,10851,13805,null,991,460,null,573,231,293,746,15482,43543,null,1,null,21,2574,7,25601,3940,10,3147,null,15980,1251,167,80972,4158
2010,522,21536,65404,26422,null,1351,40653,9401,12488,null,927,438,null,797,197,313,736,19029,43393,null,1,null,34,2545,6,22013,2485,7,3538,null,14274,1115,189,76754,3704


In [12]:
crime_pivot_mean = crime_pivot.mean()

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-3071606353383139> in <module> () 
 ----> 1 crime_pivot_mean = crime_pivot . mean ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.pyc in __getattr__ (self, name) 
 1243 if name not in self . columns : 
 1244 raise AttributeError(
 -> 1245 "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
 1246 jc = self . _jdf . apply ( name ) 
 1247 return Column ( jc ) 

 AttributeError : 'DataFrame' object has no attribute 'mean'